In [3]:
import tensorflow as tf
import numpy as np
from urllib.request import urlopen

In [4]:
# loading file
filename = "../sherlock.txt"
raw_text = open(filename).read()

In [5]:
# create mapping of unique chars of integers
chars = sorted(set(raw_text))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
# total sizes
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total characters", n_chars)
print("Total vocabulary", n_vocab)

Total characters 581862
Total vocabulary 88


In [7]:
# preparing dataset
seq_length = 300
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i : i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total patters", n_patterns)

Total patters 581562


In [8]:
# reshape X to be [samples, time steps, features]
X = tf.reshape(dataX, (n_patterns, seq_length, 1))
X = X / n_vocab
# one hot encode the output variable
y = tf.keras.utils.to_categorical(dataY)
y = tf.convert_to_tensor(y, dtype=tf.int32)

In [37]:
# parameters
dataset = n_chars
output_dim = n_vocab
EPOCHS = 500
BATCH_SIZE = 1000
learning_rate = 0.0001
patience = 10000

In [ ]:
# creating the variables of the dataset
dX = tf.placeholder(dtype=tf.float32, shape=[None, 100, 1])
dY = tf.placeholder(dtype=tf.float32, shape=[None, 85])

In [ ]:
# implementing RNN
from tensorflow.contrib import rnn
def RNN(x, rnn_size=256, dropout=0.2, batch_size=64):
    with tf.variable_scope('RNN'):
        cell = rnn.BasicRNNCell(rnn_size, activation=tf.nn.relu)
        if dropout > 0.0:
            cell = rnn.DropoutWrapper(cell, output_keep_prob= 1. - dropout)
        initial_rnn_state = cell.zero_state(batch_size, dtype=tf.float32)
        outputs, final_rnn_state = tf.nn.dynamic_rnn(cell, x, initial_state=initial_rnn_state, dtype='float32')
    outputs = tf.transpose(outputs, [1, 0, 2])
    last = outputs[-1]
    return last

In [ ]:
# implementing Softmax for Output
def softmax(input_, out_dime, scope=None):
    with tf.variable_scope(scope or 'softmax'):
        W = tf.get_variable('W', [input_.get_shape()[1], out_dim])
        b = tf.get_variable('b', [out_dim])
    return tf.nn.softmax(tf.matmul(input_, W) + b)

In [ ]:
# creaing model
logits = RNN(dX, batch_size=BATCH_SIZE)
#result = softmax(logits, output_dim)